In [13]:
def souping(URL):
    import requests
    from bs4 import BeautifulSoup
    
    HEADERS = {'User-Agent':\
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko)\
                    Chrome/47.0.2526.106 Safari/537.36'}
    HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}
    
    pageTree = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(pageTree.content, "html.parser")

    return soup


In [14]:
base= "https://www.transfermarkt.com"
url = "https://www.transfermarkt.com/fc-paris-saint-germain/spielplandatum/verein/583/saison_id//wettbewerb_id//datum_von/2019-08-01/datum_bis/2022-01-12/day/0"

In [40]:
start = souping(url)

In [15]:
import pandas as pd

In [43]:
table = start.find('div', {'class': 'responsive-table'})
df=pd.read_html(str(table))[0]
x=df
x.drop(x[x['Unnamed: 11'].map(type)!=type(float())].index, inplace=True)
x.reset_index(drop=True, inplace=True)
x.dropna(axis=1,how='all', inplace=True)
x['link'] = [i['href'].replace('index','aufstellung') for i in table.find_all('a',{'class':'ergebnis-link'})]
x

x.to_csv('psg_starting11.csv', index=False)

x = pd.read_csv('psg_starting11.csv')

In [16]:
coaches, lineups = [],[]

loadbarwidth=len(x)-1

for i in x.index:
    strbarwidth = '[{}{}] - {}\r'.format(
        (i * '#'),
        ((loadbarwidth - i) * '-'),
        (('{:0.2f}'.format(((i) * (100/loadbarwidth))) + '%'))
    )

    print(strbarwidth ,end = '')
    
    i = x.iloc[i]
    
    game=souping(base+i.link)
    lineuptables=pd.read_html(str(game.find_all('table',{'class':'items'})))
    
    if i.Venue=='H':
        lineup=lineuptables[0]
        coach =lineuptables[-3][1][0]
    else:
        lineup=lineuptables[12]
        coach=lineuptables[-1][1][0]

    players = [player.split('(')[0] for player in lineup[1]]

    lineups.append(players)
    coaches.append(coach)
    


x['coach'] = coaches
x['lineup'] = [ [player[:-1] for player in lineups[i]] for i in range(len(lineups))]

x.to_csv('psg_starting11.csv', index=False)

import pandas as pd
x = pd.read_csv('psg_starting11.csv')
x

In [108]:
import collections

In [76]:
#check consecutive lineups
for i in range(len(x)-1):
    if collections.Counter(x.lineup[i])==collections.Counter(x.lineup[i+1]):
        print(i)
print('done')

done


In [107]:
#check any lineup
for i in range(len(x)-1):
    for j in range(i+1, len(x)-1):
        if set(x.lineup_sorted[i])==set(x.lineup_sorted[j]):
            print([[i,j]])
    print(f'\r{i/(len(x)-1)*100:0.2f}%', end='')
print('\ndone')

3.73%[[6, 10]]
16.42%[[23, 25]]
24.63%[[34, 35]]
99.25%
done


In [106]:
(x.lineup_sorted[34])==( x.lineup_sorted[35])

False

In [109]:
x.Date[23],x.Date[25]

('So. 15.12.19', 'Sa. 21.12.19')